### 6.02. Reading and Writing JSON Data

In [2]:
import json

data = {
    'name' : 'ACME',
    'shares' : 100,
    'price' : 542.23,
}

json_str = json.dumps(data)
print(json_str)

{"name": "ACME", "shares": 100, "price": 542.23}


In [3]:
data = json.loads(json_str)

In [5]:
# Writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)
    
    
# Reading data back
with open('data.json', 'r') as f:
    data = json.load(f)

print(data)

{'name': 'ACME', 'shares': 100, 'price': 542.23}


In [6]:
json.dumps(False)

'false'

In [7]:
d = {
     'a': True,
     'b': 'Hello',
     'c': None
}

json.dumps(d)

'{"a": true, "b": "Hello", "c": null}'

In [ ]:
# TODO: error and fix later
# Python 3

from urllib.request import urlopen
from pprint import pprint
import json

# u = urlopen('https://search.twitter.com/search.json?q=python&rpp=5')  # error
u = urlopen('https://dev.twitter.com/docs/api/1.1/overview')  # error
u
# resp = json.loads(u.read().decode('utf-8'))
# pprint(resp)

In [71]:
from collections import OrderedDict

s = '{"name": "ACME", "shares": 50, "price": 490.1}'
data = json.loads(s, object_pairs_hook=OrderedDict)
print('data-1:', data)
print('data-2:', { k:v for k,v in data.items()})
data = json.loads(s)
print('data-3:', data)

data-1: OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
data-2: {'name': 'ACME', 'shares': 50, 'price': 490.1}
data-3: {'name': 'ACME', 'shares': 50, 'price': 490.1}


In [25]:
import json
class JSONObject():
    def __init__(self, d):
        self.__dict__ = d

def object_to_dict(obj):
    if isinstance(obj, JSONObject):
        return obj.__dict__
    raise TypeError("Type not serializable")

# Convert a JSON string to a Python object
s = '{"name": "ACME", "shares": 50, "price": 490.1}'
data = json.loads(s, object_hook=JSONObject)

print(type(data), data, data.name, data.shares, data.price, sep=', ')

print(json.dumps(data, default=object_to_dict))

<class '__main__.JSONObject'>, <__main__.JSONObject object at 0x10c8d33a0>, ACME, 50, 490.1
{"name": "ACME", "shares": 50, "price": 490.1}


In [26]:
class JSONObject():
    def __init__(self, d):
        self.__dict__ = d
        print('0.JSON:', json.dumps(self.__dict__, indent=4))
    
    def __repr__(self):
        # return str(self.__dict__)
        return 'repr: ' + json.dumps(self.__dict__, indent=4)
    
    def __str__(self):
        return 'str: ' + str(self.__dict__)
    
    def toJSON(self):
        return 'toJSON: ' + json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=2)
    
d = '{"name": "ACME", "shares": 50, "price": 490.1}'
data = json.loads(d, object_hook=JSONObject)

print('1.JSON: {!r}'.format(data))  # Calls __repr__
print('2.JSON:', data)              # Calls __str__

print('3.JSON:', json.dumps(data, indent=4, default=lambda o: o.__dict__))

print('4.JSON:', data.toJSON())


0.JSON: {
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}
1.JSON: repr: {
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}
2.JSON: str: {'name': 'ACME', 'shares': 50, 'price': 490.1}
3.JSON: {
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}
4.JSON: toJSON: {
  "name": "ACME",
  "price": 490.1,
  "shares": 50
}


In [27]:
data = json.loads(s, object_hook=JSONObject)
print(data.name, data.shares, data.price)

0.JSON: {
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}
ACME 50 490.1


In [29]:
print(data)
print(data.toJSON())
print(json.dumps(data, default=lambda o: o.__dict__, indent=2))

str: {'name': 'ACME', 'shares': 50, 'price': 490.1}
toJSON: {
  "name": "ACME",
  "price": 490.1,
  "shares": 50
}
{
  "name": "ACME",
  "shares": 50,
  "price": 490.1
}


490.1

In [30]:
print(json.dumps(data, default=lambda o: o.__dict__))

{"name": "ACME", "shares": 50, "price": 490.1}


In [31]:
print(json.dumps(data, default=lambda o: o.__dict__, indent=4))

{
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}


In [33]:
data.age = 30
print(json.dumps(data, default=lambda o: o.__dict__, sort_keys=True))

{"age": 30, "name": "ACME", "price": 490.1, "shares": 50}


In [35]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(2, 3)
json.dumps(p, default=lambda o: o.__dict__)

'{"x": 2, "y": 3}'

In [36]:
def serialize_instance(obj):
    d = { '__classname__' : type(obj).__name__ }
    d.update(vars(obj))
    return d

In [37]:
# Dictionary mapping names to known classes
classes = {
    'Point' : Point
}


def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        obj = cls.__new__(cls) # Make instance without calling __init__
        for key, value in d.items():
            setattr(obj, key, value)
            return obj
    else:
        return d

In [42]:
p = Point(2,3)
s = json.dumps(p, default=serialize_instance)
s

'{"__classname__": "Point", "x": 2, "y": 3}'

In [44]:
a = json.loads(s, object_hook=unserialize_object)
print(json.dumps(a, default=serialize_instance))

try:
    print(a.x)
    print(a.y)
except Exception as e:
    print('Error:', e)
finally:
    print('Done')


{"__classname__": "Point", "x": 2}
2
Error: 'Point' object has no attribute 'y'
Done


In [58]:
print('Hello World!')

Hello World!
